In [1]:
# global variable造成麻煩
p = 2

function pow_array(x::Vector{Float64})
    s = 0.0
    for y in x
        s = s + y^p
    end
    return s
end

pow_array (generic function with 1 method)

In [2]:
using BenchmarkTools
t = rand(100_000)

# benchmarking時用錢字號$, 以忽略呼叫t的時間, 只專注在函數的執行時間
@btime pow_array($t)  

  4.077 ms (300000 allocations: 4.58 MiB)


33202.340183578905

In [3]:
# 因為使用外部變數, 函數本身無法推測s會是什麼型別, 記憶體用量過大
@code_warntype pow_array(t)

Variables
  #self#::Core.Compiler.Const(pow_array, false)
  x::Array{Float64,1}
  s::Any
  @_4::Union{Nothing, Tuple{Float64,Int64}}
  y::Float64

Body::Any
1 ─       (s = 0.0)
│   %2  = x::Array{Float64,1}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Any
│   %11 = (y ^ Main.p)::Any
│         (s = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s


In [4]:
# 一種方式是改成const
const p2 = 2
function pow_array2(x::Vector{Float64})
    s = 0.0
    for y in x
        s = s + y^p2
    end
    return s
end

pow_array2 (generic function with 1 method)

In [5]:
@btime pow_array2($t)

  114.499 μs (0 allocations: 0 bytes)


33202.340183578905

In [6]:
# 這次回傳值的型別被推測是Float64
@code_warntype pow_array2(t)

Variables
  #self#::Core.Compiler.Const(pow_array2, false)
  x::Array{Float64,1}
  s::Float64
  @_4::Union{Nothing, Tuple{Float64,Int64}}
  y::Float64

Body::Float64
1 ─       (s = 0.0)
│   %2  = x::Array{Float64,1}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Float64
│   %11 = (y ^ Main.p2)::Float64
│         (s = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s


In [8]:
# 另一種方法是把global variable放在arguement
function pow_array3(x::Vector{Float64})
    return pow_array_inner(x, p)
end

function pow_array_inner(x, pow)
    s = 0.0
    for y in x
        s = s + y^pow
    end
    return s
end

pow_array_inner (generic function with 1 method)

In [9]:
# 這方法雖然沒有比const快, 但在某些情況不失為一種好辦法
@btime pow_array3($t)

  544.299 μs (1 allocation: 16 bytes)


33202.340183578905

In [1]:
# inline是少數在經過LLVM編譯之前, 先直接由Julia編譯器執行的優化
function f(x)
    a = 5x
    b = a + 3
end
g(x) = f(2x)

g (generic function with 1 method)

In [3]:
# 編譯器有時會自行判斷使用inline
# 例如看看這裡的Abstract Syntax Tree
# 函數g直接沿用了函數f內部的指令, 而不是額外再呼叫f
@code_typed g(3)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = Base.mul_int(5, %1)::Int64
│   %3 = Base.add_int(%2, 3)::Int64
└──      return %3
) => Int64

In [4]:
# LLVM code %1的地方乘了10, 來自g函數乘2, f函數乘5
@code_llvm g(3)


;  @ In[1]:5 within `g'
; Function Attrs: uwtable
define i64 @julia_g_20083(i64) #0 {
top:
; ┌ @ In[1]:2 within `f'
; │┌ @ int.jl:54 within `*'
    %1 = mul i64 %0, 10
; │└
; │ @ In[1]:3 within `f'
; │┌ @ int.jl:53 within `+'
    %2 = add i64 %1, 3
; └└
  ret i64 %2
}


In [5]:
# 換一個相對複雜的f函數, 而沿用同樣的g函數的定義, 這裡系統將不自動做inline
function f(x)
    a = 5x
    b = a + 3
    c = a - 4
    if c < 0
        throw(DomainError())
    elseif c < 2
        d = c^3
    else
        d = c^2
    end
end     

f (generic function with 1 method)

In [6]:
# 這裡乘完2之後就invoke f函數了
@code_typed g(3)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = invoke Main.f(%1::Int64)::Int64
└──      return %2
) => Int64

In [7]:
# 在函數前面宣告@inline, 代表我們強烈建議編譯器做inline
# 但記得最後的決定權還是在編譯器手上
@inline function f_in(x)
    a = 5x
    b = a + 3
    c = a - 4
    if c < 0
        throw(DomainError())
    elseif c < 2
        d = c^3
    else
        d = c^2
    end
end 
g_in(x) = f_in(2x)

g_in (generic function with 1 method)

In [8]:
# inline會讓程式碼變長, 但有時可能因迴圈需要跑很多次f函數, 這樣用inline還是值得的
@code_typed g_in(3)

CodeInfo(
1 ─ %1  = Base.mul_int(2, x)::Int64
│   %2  = Base.mul_int(5, %1)::Int64
│   %3  = Base.sub_int(%2, 4)::Int64
│   %4  = Base.slt_int(%3, 0)::Bool
└──       goto #3 if not %4
2 ─       Main.DomainError()::Union{}
└──       $(Expr(:unreachable))::Union{}
3 ┄ %8  = Base.slt_int(%3, 2)::Bool
└──       goto #5 if not %8
4 ─ %10 = Base.mul_int(%3, %3)::Int64
│   %11 = Base.mul_int(%10, %3)::Int64
└──       goto #6
5 ─ %13 = Base.mul_int(%3, %3)::Int64
└──       goto #6
6 ┄ %15 = φ (#4 => %11, #5 => %13)::Int64
└──       return %15
) => Int64

In [36]:
# 有時為了debug, 我們會希望不要inline, 即使是編譯器自動幫我們做的
# 這時候需要@noinline
@noinline function f_ni(x)
    a = 5x
    b = a + 3
end

g_ni(x) = f_ni(2x)

g_ni (generic function with 1 method)

In [37]:
@code_typed g(3)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = invoke Main.f(%1::Int64)::Int64
└──      return %2
) => Int64

In [1]:
# constant propagation
# 一個純淨的函數被以某個arguement作呼叫, 則會以定值被存取, 不會再重複調用
# 純淨指的是不影響環境中其他變數或狀態為原則
sqr(x) = x * x
sqr2() = sqr(2)

sqr2 (generic function with 1 method)

In [2]:
# 因此多利用純淨的函數, 可以讓編譯器有更多機會優化我們的程式
@code_typed sqr2()

CodeInfo(
1 ─     return 4
) => Int64

In [7]:
# 計算 p(x) = a₀ + a₁x + a₂x² + ... + aₙxⁿ

In [8]:
# 傳統方法
function poly_naive(x, a...)
    p = zero(x)
    for i in 1:length(a)
        p = p + a[i] * x^(i-1)
    end
    return p
end

poly_naive (generic function with 1 method)

In [9]:
f_naive(x) = poly_naive(x, 1,2,3,4,5,6,7,8,9)

f_naive (generic function with 1 method)

In [13]:
# 時間比想像中花得還要久, 原因是浮點數的次方運算往往比較複雜
# 況且這個算法花了太多餘的計算量, 每次都重算更高的次方
x = 3.5
@btime f_naive($x)

  350.467 ns (0 allocations: 0 bytes)


271125.95703125

In [14]:
# Horner's method
function poly_honer(x, a...)
    b = zero(x)
    for i = length(a):-1:1
        b = a[i] + b * x
    end
    return b
end

poly_honer (generic function with 1 method)

In [15]:
f_horner(x) = poly_honer(x, 1,2,3,4,5,6,7,8,9)

f_horner (generic function with 1 method)

In [16]:
@btime f_horner($x)

  3.899 ns (0 allocations: 0 bytes)


271125.95703125

In [17]:
# 上面的程式碼想做的事, 事實上就是用muladd()函數層層包起來
# muladd(x, muladd(x, muladd(x, ..., 3), 2), 1)
# 這種寫這種重複的程式碼, 且要兼顧泛用性, 最好的方式就是用macro
# macro是用來寫程式的函數, 是屬於meta-programming的部分
# 實作就是將每個muladd()表達式包起來, 最後再一次執行, 這樣會有最好的效率
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :(muladd(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

In [19]:
f_horner_macro(x) = @horner(x, 1,2,3,4,5,6,7,8,9)

f_horner_macro (generic function with 1 method)

In [20]:
@btime f_horner_macro($x)

  0.001 ns (0 allocations: 0 bytes)


271125.95703125

In [1]:
# 有時我們會想要有macro的便利, 又希望它能針對型別作處理
# 這時候就需要用 generated function

In [5]:
# 舉個例子, 如果我們想計算多維陣列的元素個數, 可能會這麼寫
function prod_dim(x::Array{T, N}) where {T, N}
    s = 1
    for i = 1:N
        s = s * size(x, i)
    end
    return s
end

prod_dim (generic function with 1 method)

In [6]:
prod_dim(rand(10, 5, 5))

250

In [12]:
# 觀察: 陣列的維度也是其中一個型別, 如果型別固定, 迴圈的內容就是固定的
# 所以可以試著寫一個對某個型別的泛用函數, 來做像macro一樣的事, code that generates code
@generated function prod_dim_gen(x::Array{T, N}) where {T, N}
    ex = :(1)
    for i in 1:N
        ex = :(size(x, $i) * $ex)
    end
    return ex
end

prod_dim_gen (generic function with 1 method)

In [13]:
prod_dim_gen(rand(10, 5, 5))

250

In [16]:
# 若用一般的函數來寫, 看結果會長怎麼樣
function prod_dim_gen_impl(x::Array{T, N}) where {T, N}
    ex = :(1)
    for i in 1:N
        ex = :(size(x, $i) * $ex)
    end
    return ex
end

prod_dim_gen_impl (generic function with 1 method)

In [17]:
x = rand(10, 5, 5);
prod_dim_gen_impl(x)

:(size(x, 3) * (size(x, 2) * (size(x, 1) * 1)))

In [18]:
# 這便是傳給Julia編譯器的真正的程式碼
# macro的執行順序是很前面的, 幾乎是剛從硬碟讀程式檔後就最先執行的動作
# 而且一種型別只有第一次生成的時候需要引用上面的迴圈, 之後每次遇到就能重複使用, 更不消耗效能
x = rand(10, 5, 5, 2);
prod_dim_gen_impl(x)

:(size(x, 4) * (size(x, 3) * (size(x, 2) * (size(x, 1) * 1))))

In [19]:
# 這種方法可以消除不必要的迴圈, 是處理效能問題重要的招式
# 再搭配inline的做法, 讓我們花費的時間可以顯著減少